In [1]:
from user_ops import ft_inverse
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import math
import pickle
import json

import os
import sys
sys.path.append(os.path.join('..', '..', 'keras_frac'))
from fractional_maxpooling import FractionalPooling2D
import resnet

%matplotlib inline

init = "he_uniform"

/opt/miniconda3/envs/tf1_13/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/miniconda3/envs/tf1_13/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/miniconda3/envs/tf1_13/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/miniconda3/envs/tf1_13/lib/python3.7/site-packages/tensorflow/

In [2]:
class max_pool_net:
    def __init__(self, opt, batch_input_shape, classes, k_size=(3,3), filters=2, blocks=2):
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.InputLayer(batch_input_shape=batch_input_shape))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='r2c2elu', padding='same', kernel_initializer=init))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same', kernel_initializer=init))
        self.model.add(keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
        for i in range(2, blocks+1):
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same', kernel_initializer=init))
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same', kernel_initializer=init))
            self.model.add(keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
        self.model.add(keras.layers.Flatten())
        self.model.add(keras.layers.Dense(classes, activation='softmax'))
        self.model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
        print(self.model.summary())
    
    def train(self, batch_size, epochs, datagen=None, train_data=None, callbacks=None):
        if datagen is None and train_data is None:
            print('neither data or generator was passed')
        elif datagen is None and train_data is not None:
            print('training on array data, network type:', type(self).__name__)
            self.history = self.model.fit(x = train_data[0], y = train_data[1], validation_data=train_data[2:], batch_size=batch_size, epochs=epochs, callbacks=callbacks)
        else:
            print('training on datagen data, network type:', type(self).__name__)
            self.history = self.model.fit_generator(datagen, validation_data=train_data[2:], batch_size=batch_size, epochs=epochs, callbacks=callbacks)
        self.weights = self.model.get_weights()
    
    def restart_session(self):
        keras.backend.clear_session()
        
    def get_history(self):
        return self.history
    
    def get_weights(self):
        return self.weights
    
    def load_weights(self):
        self.model.load_weights(self.weights)

        
class frac_pool_net(max_pool_net):
    def __init__(self, opt, batch_input_shape, classes, k_size=(3,3), filters=2, blocks=2):
        ratio = 1.35 # 1.33 blocks=3
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.InputLayer(batch_input_shape=batch_input_shape))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same', kernel_initializer=init))
        self.model.add(FractionalPooling2D(pool_ratio=(1, ratio, ratio, 1),pseudo_random = True, overlap=False))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same', kernel_initializer=init))
        self.model.add(FractionalPooling2D(pool_ratio=(1, ratio, ratio, 1),pseudo_random = True, overlap=False))
        for i in range(2, blocks+1):
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same', kernel_initializer=init))
            self.model.add(FractionalPooling2D(pool_ratio=(1, ratio, ratio, 1),pseudo_random = True, overlap=False))
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same', kernel_initializer=init))
            self.model.add(FractionalPooling2D(pool_ratio=(1, ratio, ratio, 1),pseudo_random = True, overlap=False))
        self.model.add(keras.layers.Flatten())
        self.model.add(keras.layers.Dense(classes, activation='softmax'))
        self.model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
        print(self.model.summary())
    
    
class ft_pool_net(max_pool_net):
    def __init__(self, opt, batch_input_shape, classes, k_size=(3,3), filters=2, blocks=2):
        strides=(math.sqrt(2), math.sqrt(2))
        pool_size=(math.sqrt(2)*2, math.sqrt(2)*2)
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.InputLayer(batch_input_shape=batch_input_shape))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same', kernel_initializer=init))
        self.model.add(keras.layers.Lambda(lambda x: ft_pool(x, strides, pool_size)))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same', kernel_initializer=init))
        self.model.add(keras.layers.Lambda(lambda x: ft_pool(x, strides, pool_size)))
        for i in range(2, blocks+1):
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same', kernel_initializer=init))
            self.model.add(keras.layers.Lambda(lambda x: ft_pool(x, strides, pool_size)))
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same', kernel_initializer=init))
            self.model.add(keras.layers.Lambda(lambda x: ft_pool(x, strides, pool_size)))
        self.model.add(keras.layers.Flatten())
        self.model.add(keras.layers.Dense(classes, activation='softmax'))
        self.model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
        print(self.model.summary())
        
        
class avg_pool_net(max_pool_net):
    def __init__(self, opt, batch_input_shape, classes, k_size=(3,3), filters=2, blocks=2):
        pool_size=2
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.InputLayer(batch_input_shape=batch_input_shape))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same', kernel_initializer=init))
        self.model.add(keras.layers.Conv2D(filters, k_size, activation='relu', padding='same', kernel_initializer=init))
        self.model.add(keras.layers.AveragePooling2D(pool_size=(pool_size, pool_size), strides=(2,2), padding='same'))
        for i in range(2, blocks+1):
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same', kernel_initializer=init))
            self.model.add(keras.layers.Conv2D(filters*i, k_size, activation='relu', padding='same', kernel_initializer=init))
            self.model.add(keras.layers.AveragePooling2D(pool_size=(pool_size, pool_size), strides=(2,2), padding='same'))
        self.model.add(keras.layers.Flatten())
        self.model.add(keras.layers.Dense(classes, activation='softmax'))
        self.model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])
        print(self.model.summary())

In [3]:
EPOCHS = 30
BATCH_SIZE = 100
iter = 1

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [5]:
ft_net_h = []
path = os.path.join('results', 'mine_with_div')
for _ in range(iter):
    ft_net = ft_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
    ft_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
    ft_net_h.append([ft_net.get_history().history['accuracy'], ft_net.get_history().history['val_accuracy']])
    ft_net.restart_session()
with open(os.path.join(path,'ft_net_h.pkl'), 'wb') as f:
    pickle.dump(ft_net_h, f)
'''
avg_net_h = []
for _ in range(iter):
    avg_net = avg_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
    avg_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
    avg_net_h.append([avg_net.get_history().history['accuracy'], avg_net.get_history().history['val_accuracy']])
    avg_net.restart_session()
with open(os.path.join(path,'avg_net_h.pkl'), 'wb') as f:
    pickle.dump(avg_net_h, f)

max_net_h = []
for _ in range(iter):
    max_net = max_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
    max_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
    max_net_h.append([max_net.get_history().history['accuracy'], max_net.get_history().history['val_accuracy']])
    max_net.restart_session()
with open(os.path.join(path,'max_net_h.pkl')) as f:
    pickle.dump(max_net_h, f)

frac_net_h = []
for _ in range(iter):
    frac_net = frac_pool_net(keras.optimizers.Adadelta(), (BATCH_SIZE,) + x_train.shape[1:], y_train.shape[1], blocks=2)
    frac_net.train(BATCH_SIZE, EPOCHS, train_data=(x_train, y_train, x_test, y_test), callbacks=[keras.callbacks.ReduceLROnPlateau(patience=5, factor=.5, verbose=1)])
    frac_net_h.append([frac_net.get_history().history['accuracy'], frac_net.get_history().history['val_accuracy']])
    frac_net.restart_session()
with open(os.path.join(path,'frac_net_h.pkl'), 'wb') as f:
    pickle.dump(frac_net_h, f)
'''

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (100, 32, 32, 2)          56        
_________________________________________________________________
lambda_1 (Lambda)            (100, 23, 23, 2)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (100, 23, 23, 2)          38        
_________________________________________________________________
lambda_2 (Lambda)            (100, 16, 16, 2)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (100, 16, 16, 4)          76        
_________________________________________________________________
lambda_3 (Lambda)            (100, 11, 11, 4)          0         
________________________________________________

InvalidArgumentError: Input to reshape is a tensor with 6656400 values, but the requested shape requires a multiple of 256
	 [[{{node flatten_1/Reshape}}]]

In [ ]:
# baseline best 2x max pool = 53.19%

In [ ]:

with open(os.path.join('/mnt/F0CC2854CC28177E/repos/workIRAFM/tensorflow-cmake/custom_op/results/mine_without_div','max_net_h.pkl'), 'rb') as f:
    max_net_h = pickle.load(f)
plt.figure(figsize=(15,7))
arr = np.array(max_net_h)
plt.plot(np.mean(arr[:,0], axis=0), 'r')
plt.plot(np.max(arr[:,0], axis=0), 'r--')

with open(os.path.join('/mnt/F0CC2854CC28177E/repos/workIRAFM/tensorflow-cmake/custom_op/results/mine_without_div','frac_net_h.pkl'), 'rb') as f:
    frac_net_h = pickle.load(f)
plt.figure(figsize=(15,7))
arr = np.array(frac_net_h)
plt.plot(np.mean(arr[:,0], axis=0), 'g')
plt.plot(np.max(arr[:,0], axis=0), 'g--')
with open(os.path.join('/mnt/F0CC2854CC28177E/repos/workIRAFM/tensorflow-cmake/custom_op/results/mine_without_div','ft_net_h.pkl'), 'rb') as f:
    ft_net_h = pickle.load(f)
plt.figure(figsize=(15,7))
arr = np.array(ft_net_h)
plt.plot(np.mean(arr[:,0], axis=0), 'b')
plt.plot(np.max(arr[:,0], axis=0), 'b--')
with open(os.path.join('/mnt/F0CC2854CC28177E/repos/workIRAFM/tensorflow-cmake/custom_op/results/mine_without_div','avg_net_h.pkl'), 'rb') as f:
    avg_net_h = pickle.load(f)
plt.figure(figsize=(15,7))
arr = np.array(avg_net_h)
plt.plot(np.mean(arr[:,0], axis=0), 'c')
plt.plot(np.max(arr[:,0], axis=0), 'c--')

In [ ]:
with open(os.path.join('/mnt/F0CC2854CC28177E/repos/workIRAFM/tensorflow-cmake/custom_op/results/mine_without_div','max_net_h.pkl'), 'rb') as f:
    max_net_h = pickle.load(f)
plt.figure(figsize=(15,7))
arr = np.array(max_net_h)
arr = np.mean(arr[:,1], axis=0)
plt.plot(arr, 'r')
for v in range(len(arr)):
  print("({},{:.3f})".format(v, arr[v]), end=' ')
print()

with open(os.path.join('/mnt/F0CC2854CC28177E/repos/workIRAFM/tensorflow-cmake/custom_op/results/mine_without_div','frac_net_h.pkl'), 'rb') as f:
    frac_net_h = pickle.load(f)
arr = np.array(frac_net_h)
arr = np.mean(arr[:,1], axis=0)
plt.plot(arr, 'g')
for v in range(len(arr)):
  print("({},{:.3f})".format(v, arr[v]), end=' ')
print()

with open(os.path.join('/mnt/F0CC2854CC28177E/repos/workIRAFM/tensorflow-cmake/custom_op/results/mine_without_div','ft_net_h.pkl'), 'rb') as f:
    ft_net_h = pickle.load(f)
arr = np.array(ft_net_h)
arr = np.mean(arr[:,1], axis=0)
plt.plot(arr, 'b')
for v in range(len(arr)):
  print("({},{:.3f})".format(v, arr[v]), end=' ')
print()

with open(os.path.join('/mnt/F0CC2854CC28177E/repos/workIRAFM/tensorflow-cmake/custom_op/results/mine_without_div','avg_net_h.pkl'), 'rb') as f:
    avg_net_h = pickle.load(f)
arr = np.array(avg_net_h)
arr = np.mean(arr[:,1], axis=0)
plt.plot(arr, 'c')
for v in range(len(arr)):
  print("({},{:.3f})".format(v, arr[v]), end=' ')
print()

In [ ]:
ftpoolresnet = resnet.FtPoolingResnet(n=3)
ftpoolresnet.build_network()
ftpoolresnet.train()
ftpoolresnet.save_history('resnet')

In [ ]:
for v in range(len(ftpoolresnet.history.history['val_accuracy'])):
  print("({},{:.3f})".format(v, ftpoolresnet.history.history['val_accuracy'][v]), end=' ')

In [ ]:
with open("/mnt/F0CC2854CC28177E/repos/workIRAFM/lenetft/ftnet_with_permutations/history_jsons_8_kernels/resnet20.json") as f:
    data = json.load(f)

In [ ]:
_sum = np.zeros(200, dtype=np.float32)
for s in data['val_accuracy']:
    _sum += np.array(s)
_sum /= 20

In [ ]:
for v in range(len(_sum)):
  print("({},{:.3f})".format(v, _sum[v]), end=' ')